# Q-Learning

In [1]:
# After we publish the pack age, we can directly import it
# TODO: explore more efficient way
# we can hide this cell later
import os
os.getcwd()
os.chdir('..')
os.chdir('../CausalDM')

## Main Idea

Q-learning is a classic method of Reinforcement Learning. Early in 2000, it was adapted to decision-making problems[1] and kept evolving with various extensions, such as penalized Q-learning [2]. In the following, we would start from a simple case having only one decision point, and then introduce the multistage case with multiple decision points. Note that, we assume the action space is either **binary** (i.e., 0,1) or **multinomial** (i.e., A,B,C,D), and the outcome of interest Y is **continuous** and **non-negative**, where the larger the $Y$ the better. 

### Single Decision Point

- **Application Situation**: Suppose we have a dataset contains observations from $N$ individuals. For each individual $i$, we have $\{\mathbf{X}_{i},A_{i},Y_{i}\}$, $i=1,\cdots,N$. $\mathbf{X}_{i}$ includes the feature information, $A_{i}$ is the action taken, and $Y_{i}$ is the observed reward received. The target of Q-learing is to find an optimal policy $\pi$ that can maximize the expected reward received. In other words, training a model with the observed dataset, we want to find an optimal policy that can help us determine the optimal action for each individual to optimize the reward.

- **Basic Logic**: Q-learning with single decision point is mainly a regression modeling problem, as the major component is to find the relationship between $Y$ and $\{X,A\}$. Let's first define a Q-function, such that
\begin{align}
    Q(x,a) = E(Y|X=x, A=a).
\end{align} Then, to find the optimal policy is equivalent to solve
\begin{align}
    \text{arg max}_{\pi}Q(x_{i},\pi(x_{i})).
\end{align} 

- **Key Steps**:
    1. Fitted a model $\hat{Q}(x,a,\hat{\beta})$, which can be solved directly by existing approaches (i.e., OLS, .etc),
    2. For each individual find the optimal action $a_{i}$ such that $a_{i} = \text{arg max}_{a}\hat{Q}(x_{i},a,\hat{\beta})$.

### Multiple Decision Points

- **Application Situation**: Suppose we have a dataset contains observations from $N$ individuals. For each individual $i$, the observed data is structured as follows
\begin{align}
(X_{1i},A_{1i},\cdots,X_{Ti},A_{Ti},Y), i=1,\cdots, N.
\end{align} Let $h_{ti}=\{X_{1i},A_{1i},\cdots,X_{ti},A_{ti}\})$ includes all the information observed till step t. The target of Q-learing is to find an optimal policy $\pi$ that can maximize the expected reward received at the end of the final decision point $T$. In other words, training a model with the observed dataset, we want to find an optimal policy that can help us determine the optimal sequence of actions for each individual to optimize the reward.

- **Basic Logic**: For the multistage case, we utilize a backward iterative framework, which means that we start from the final decision point T and then gioing backwards till the initial decision point. At the final step $T$, it is a standard regression modeling problem that is exactly the same as what we did for the single decision point case. Particularly, we posit a model $Q_{T}(h_{T},a_{T})$ for the observed outcome $Y$, and then the optimal policy at step $T$ is derived as $\text{arg max}_{\pi_{T}}Q_{T}(h_{T},\pi_{T}(h_{T}))$. For the decision point $T-1$ till the decision point $1$, a new term is introduced, which is the pseudo-outcome $\tilde{Y}_{t}$.
\begin{align}
\tilde{Y}_{t} = \text{max}_{\pi_{t}}\hat{Q}_{t}(h_{t},\pi_{t}(h_{t}),\hat{\beta}_{t})
\end{align} By doing so, the pseudo-outcome taking the **delayed effect** into account to help explore the optimal policy. Then, for each decision point $t<T$, with the $\tilde{Y}_{t+1}$ calculated, we repeat the regression modeling step for $\tilde{Y}_{t+1}$. After obtainning the fitted model $\hat{Q}_{t}(h_{t},a_{t},\hat{\beta}_{t})$, the optimal policy is obtained as $\text{arg max}_{\pi_{t}}Q_{t}(h_{t},\pi_{t}(h_{t}))$.

- **Key Steps**: 
    1. At the final decision point $t=T$, fitted a model $\hat{Q}_{T}(h_{T},a_{T},\hat{\beta}_{T})$ for the observed outcome $Y$;
    2. For each individual $i$, calculated the pesudo-outcome $\tilde{Y}_{Ti}=\text{max}_{\pi}\hat{Q}_{T}(h_{Ti},\pi(h_{Ti}),\hat{\beta}_{T})$, and the optimal action $a_{Ti}=\text{arg max}_{a}\hat{Q}_{T}(h_{Ti},a,\hat{\beta}_{T})$;
    3. For decision point $t = T-1,\cdots, 1$,
        1. fitted a model $\hat{Q}_{t}(h_{t},a_{t},\hat{\beta}_{t})$ for the pseudo-outcome $\tilde{Y}_{t+1}$
        2. For each individual $i$, calculated the pesudo-outcome $\tilde{Y}_{ti}=\text{max}_{\pi}\hat{Q}_{t}(h_{ti},\pi(h_{ti}),\hat{\beta}_{t})$, and the optimal action $a_{ti}=\text{arg max}_{a}\hat{Q}_{t}(h_{ti},a,\hat{\beta}_{t})$;


## Demo

In [11]:
# A demo with code on how to use the package
from causaldm.learners import QLearning
from causaldm.test import shared_simulation
import numpy as np

In [12]:
#prepare the dataset (dataset from the DTR book)
import pandas as pd
file = pd.read_csv("hyper.txt", sep=',')
file['Y'] = file['SBP0']-file['SBP6']
hyper = file
Y = hyper['Y']
X = hyper[['W','K','Cr','Ch']]
A = hyper['A']

In [15]:
# initialize the learner
QLearn = QLearning.QLearning()
# specify the model you would like to use
# If want to include all the variable in X and A with no specific model structure, then use either "Y~." or "Y~X+A"
# Otherwise, specify the model structure by hand
# Note: if the action space is not binary, use C(A) in the model instead of A
model_info = [{"model": "Y~Ch+K+A+Ch*A+K*A",
              'A': [0,1],
             'feature':X,
             'outcome':Y}]

# train the policy
QLearn.train(X, A, Y, model_info)

In [16]:
# recommend action
QLearn.recommend_action(X)[:10]

0    0
1    0
2    1
3    1
4    1
5    1
6    1
7    0
8    1
9    1
dtype: int64

In [17]:
# get the estimated value of the optimal regime
QLearn.estimate_value(X)

13.215622865312115

In [18]:
# Fitted Model
QLearn.fitted_model.params

Intercept   -15.604845
Ch           -0.203472
K            12.284852
A           -61.097909
Ch:A          0.504816
K:A          -6.609876
dtype: float64

In [ ]:
# generate sample data
from causaldm.test import OWL_simu
import pandas as pd
import statsmodels.api as sm
instance = OWL_simu.generate_test_case(setup = 'case1', N = 1000, seed = 0, p = 5, sigma = 1)
X, A, Y = instance['XAY']
X = pd.DataFrame(X);A = pd.DataFrame(A);Y = pd.DataFrame(Y)

# initialize the learner
QLearn = QLearning.QLearning()
# specify the model you would like to use
model_info = [{"model": "Y~X+A", 
              'A': [0,1],
             'feature':X,
             'outcome':Y}]

# train the policy
QLearn.train(X, A, Y, model_info)

## References
1. Murphy, S. A. (2005). A generalization error for Q-learning.
2. Song, R., Wang, W., Zeng, D., & Kosorok, M. R. (2015). Penalized q-learning for dynamic treatment regimens. Statistica Sinica, 25(3), 901.